# How do we process the data?

There are a few preset functions that already do this: the Dataloader in the Pytorch library, Keras also has it's dataloader function. However, during the course of the two use cases we explored above, we found out that we had to do extra preparation for loading. For asset price prediction, we did some extra preparation to put it into the Dataloader class from Pytorch, whereas for the quantum drug generator we instead used the dataloader function from the ReLeaSE algorithm. 

What type of data do we work with?

- Time series Data
- Natural Language Processing
- Image Captioning
- Machine Translation

In [1]:
import time
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import time
from tqdm import trange

from utils import time_since

import pennylane as qml
from quantum import QGRU

#from smiles_enumerator import SmilesEnumerator

C:\Users\jonas\Anaconda3\envs\my-rdkit-env\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [ ]:
class GeneratorData(object):
    """
    Docstring coming soon...
    """

    def __init__(self, training_data_path, tokens=None, start_token='<',
                 end_token='>', pad_symbol=' ', max_len=120, use_cuda=None,
                 **kwargs):
        """
        Constructor for the GeneratorData object.

        Parameters
        ----------
        training_data_path: str
            path to file with training dataset. Training dataset must contain
            a column with training strings. The file also may contain other
            columns.

        tokens: list (default None)
            list of characters specifying the language alphabet. Of left
            unspecified, tokens will be extracted from data automatically.

        start_token: str (default '<')
            special character that will be added to the beginning of every
            sequence and encode the sequence start.

        end_token: str (default '>')
            special character that will be added to the end of every
            sequence and encode the sequence end.

        max_len: int (default 120)
            maximum allowed length of the sequences. All sequences longer than
            max_len will be excluded from the training data.

        use_cuda: bool (default None)
            parameter specifying if GPU is used for computations. If left
            unspecified, GPU will be used if available

        kwargs: additional positional arguments
            These include cols_to_read (list, default [0]) specifying which
            column in the file with training data contains training sequences
            and delimiter (str, default ',') that will be used to separate
            columns if there are multiple of them in the file.

        """
        super(GeneratorData, self).__init__()

        if 'cols_to_read' not in kwargs:
            kwargs['cols_to_read'] = []

        data = read_object_property_file(training_data_path,
                                         **kwargs)
        self.start_token = start_token
        self.end_token = end_token
        self.pad_symbol = pad_symbol
        self.file = []
        for i in range(len(data)):
            if len(data[i]) <= max_len:
                self.file.append(self.start_token + data[i] + self.end_token)
        self.file_len = len(self.file)
        self.tokens = tokens
        self.all_characters, self.char2idx, \
        self.n_characters = tokenize(self.file, tokens)
        self.pad_symbol_idx = self.all_characters.index(self.pad_symbol)
        self.use_cuda = use_cuda
        if self.use_cuda is None:
            self.use_cuda = torch.cuda.is_available()

    def load_dictionary(self, tokens, char2idx):
        self.all_characters = tokens
        self.char2idx = char2idx
        self.n_characters = len(tokens)

    def random_chunk(self, batch_size):
        """
        Samples random SMILES string from generator training data set.
        Returns:
            random_smiles (str).
        """
        index = np.random.randint(0, self.file_len - 1, batch_size)
        return [self.file[i][:-1] for i in index], \
               [self.file[i][1:] for i in index]

    def seq2tensor(self, seqs, tokens, flip=True):
        tensor = np.zeros((len(seqs), len(seqs[0])))
        for i in range(len(seqs)):
            for j in range(len(seqs[i])):
                if seqs[i][j] in tokens:
                    tensor[i, j] = int(tokens.index(seqs[i][j]))
                else:
                    tokens = tokens + [seqs[i][j]]
                    tensor[i, j] = int(tokens.index(seqs[i][j]))
        if flip:
            tensor = np.flip(tensor, axis=1).copy()
        return tensor, tokens

    def pad_sequences(self, seqs, max_length=None, pad_symbol=' '):
        if max_length is None:
            max_length = -1
            for seq in seqs:
                max_length = max(max_length, len(seq))
        lengths = []
        for i in range(len(seqs)):
            cur_len = len(seqs[i])
            lengths.append(cur_len)
            seqs[i] = seqs[i] + pad_symbol * (max_length - cur_len)
        return seqs, lengths

    def random_training_set(self, batch_size):
        inp, target = self.random_chunk(batch_size)
        inp_padded, _ = self.pad_sequences(inp, pad_symbol=self.pad_symbol)
        inp_tensor, self.all_characters = self.seq2tensor(inp_padded,
                                                          tokens=self.all_characters,
                                                          flip=False)
        target_padded, _ = self.pad_sequences(target, pad_symbol=self.pad_symbol)
        target_tensor, self.all_characters = self.seq2tensor(target_padded,
                                                             tokens=self.all_characters,
                                                             flip=False)
        self.n_characters = len(self.all_characters)
        inp_tensor = torch.tensor(inp_tensor).long()
        target_tensor = torch.tensor(target_tensor).long()
        if self.use_cuda:
            inp_tensor = inp_tensor.cuda()
            target_tensor = target_tensor.cuda()
        return inp_tensor, target_tensor

    def read_sdf_file(self, path, fields_to_read):
        raise NotImplementedError

    def update_data(self, path):
        self.file, success = read_smi_file(path, unique=True)
        self.file_len = len(self.file)
        assert success
        
    def char_tensor(self, string):
        """
        Converts SMILES into tensor of indices wrapped into torch.autograd.Variable.
        Args:
            string (str): input SMILES string
        Returns:
            tokenized_string (torch.autograd.Variable(torch.tensor))
        """
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = self.all_characters.index(string[c])
        if self.use_cuda:
            return torch.tensor(tensor).cuda()
        else:
            return torch.tensor(tensor)